In [ ]:
# Generate Human filled label table
import pandas as pd
import spacy

nlp = spacy.load("zh_core_web_lg")
data_path = "data/question_label.csv"

ac = pd.read_csv(data_path,sep=",",header=0)

count = 0
total = 100
# using dependency as chunk_tags
with open("chunk_question.csv", 'w') as f:
    f.write("text,chunk_tags,pos_tags,ner_tags\n")
    for index, row in ac.iterrows():
        # Line Note
        if count == 0:
            count = count + 1
            continue
        # Control line
        # if count >= 3:
        #     break
        # print(row['text'])
        # print(pd.isnull(row['text']))
        if pd.isnull(row['text']):
            continue
        doc = nlp(row['text'])
        chunck_text = "" 
        chunk_tags = "" 
        pos_tags = ""
        for i, token in enumerate(doc):
            if i!=0:
                chunck_text+=" "
                chunk_tags+=" "
                pos_tags+=" "
            chunck_text+=token.text
            chunk_tags+=token.dep_
            pos_tags+=token.tag_
            
        f.write(chunck_text)
        f.write(",")
        f.write(pos_tags)
        f.write(",")
        f.write(token.pos_)
        f.write(",\n")
        count = count + 1

In [7]:
# Collection the index of pos, chunk, and ner
import pandas as pd
import spacy
import json

nlp = spacy.load("zh_core_web_lg")

data_path = "data/chunk_question_v1_1.csv"
train_path = "data/train_new1.json"
valid_path = "data/valid_new1.json"
test_path = "data/test_new1.json"

tags = {"org_text":0, "lemma_text":1, "pos_tags":2, "chunk_tags":3, "ner_tags":4}
pos_mapping = {'"': 0, "''": 1, '#': 2, '$': 3, '(': 4, ')': 5, ',': 6, '.': 7, ':': 8, '``': 9, 'NN': 10, 'KON': 11, '$.': 12, 'APPRART': 13, 'ADJA': 14, 'NE': 15, 'VVFIN': 16, 'APPR': 17, '$,': 18, 'CARD': 19, 'TRUNC': 20, 'VAFIN': 21, 'ART': 22, 'VVPP': 23, 'PPER': 24, 'PAV': 25, 'ADJD': 26, 'PTKNEG': 27, 'PTKZU': 28, 'VVINF': 29, 'ADV': 30, 'PRELS': 31, 'PRF': 32, 'PDAT': 33, 'KOUS': 34, 'PIS': 35, 'PIAT': 36, 'VMFIN': 37, 'PPOSAT': 38, 'VVIZU': 39, 'VAPP': 40, '$(': 41, 'VAINF': 42, 'PTKVZ': 43, 'PIDAT': 44, 'KOKOM': 45, 'PDS': 46, 'PTKA': 47, 'PRELAT': 48, 'PWAV': 49, 'PWAT': 50, 'KOUI': 51, 'VMINF': 52, 'PWS': 53, 'APPO': 54, 'VVIMP': 55, 'PTKANT': 56, 'FM': 57, 'APZR': 58, 'VAIMP': 59, 'ITJ': 60, 'PPOSS': 61, 'XY': 62, 'VMPP': 63, 'NR': 64, 'AD': 65, 'CC': 66, 'VC': 67, 'DT': 68, 'DEC': 69, 'DEG': 70, 'P': 71, 'VV': 72, 'CD': 73, 'JJ': 74, 'M': 75, 'OD': 76, 'PU': 77, 'NT': 78, 'LC': 79, 'PN': 80, 'VA': 81, 'VE': 82}
pos_count = 83 
chunk_mapping = {'O': 0, 'compound:nn': 1, 'nsubj': 2, 'ROOT': 3, 'dep': 4, 'conj': 5, 'cc': 6, 'cop': 7, 'appos': 8, 'det': 9, 'nmod:assmod': 10, 'case': 11, 'nmod': 12, 'dobj': 13, 'amod': 14, 'advmod': 15, 'mark:clf': 16, 'acl': 17, 'mark': 18, 'nummod': 19, 'nmod:prep': 20, 'nmod:range': 21, 'nmod:tmod': 22, 'amod:ordmod': 23, 'punct': 24, 'ccomp': 25, 'advmod:rcomp': 26, 'advmod:loc': 27, 'name': 28}
chunk_count = 29 
ner_mapping = {"O":0, "I-Column": 1, "B-Column":2, "I-Operation":3, "B-Operation":4, "I-Restriction":5, "B-Restriction":6, "I-Pattern":7, "B-Pattern":8, "I-Others":9, "B-Others": 10}
ner_count = 9

dataframe = pd.read_csv(data_path,sep=',',header=0)
dataframe.head()

with open(valid_path,'w',encoding='utf-8') as valid:
    with open(test_path,'w',encoding='utf-8') as test:
        with open(train_path,'w',encoding='utf-8') as train:
            for index, row in dataframe.iterrows():
                # if index >= 1:
                #     break
                text = row["text"].replace(' ','')
                doc = nlp(text)
                
                result = {}
                result["id"] = str(index)
                result["chunk_tags"] = []
                result["ner_tags"] = []
                result["pos_tags"] = []
                result["tokens"] = []

                chunk_tags = []
                ner_tags = []
                pos_tags = []
                tokens = []
                for i, token in enumerate(doc):
                    # print(f"token.dep_:{token.dep_}") # chunk tag

                    # collect chunk tag
                    chunk_tag = token.dep_
                    chunk_tags.append(chunk_mapping[chunk_tag])
                    
                    # # collect unknown chunk tag
                    # if chunk_tag not in chunk_mapping:
                    #     chunk_mapping[chunk_tag] = chunk_count
                    #     chunk_count = chunk_count + 1
                    # print(f"token.tag_:{token.tag_}") # pos tag

                    # collect pos tag
                    pos_tag = token.tag_
                    pos_tags.append(pos_mapping[pos_tag])

                    # # collect unknown pos tag
                    # if pos_tag not in pos_mapping:
                    #     pos_mapping[pos_tag] = pos_count
                    #     pos_count = pos_count + 1

                    # collect ner tag
                    ner_tags_list = row["ner_tags"].split()
                    e_ner_tag = int(ner_tags_list[i])
                    # skip column name
                    if e_ner_tag == 1 or e_ner_tag == 2:
                        ner_tags.append(0)
                    else:
                        ner_tags.append(e_ner_tag)

                    # collect tokens
                    # print(token.text)
                    tokens.append(token.text)

                # Debug
                # if index > 176:
                #     print(f"len_ner_tags:{len(ner_tags)}")
                #     print(f"len_origin_text:{(row['text'].split())}")
                #     print(f"tokens_origin_text:{(tokens)}")

                result["chunk_tags"] = chunk_tags 
                result["ner_tags"] = ner_tags 
                result["pos_tags"] = pos_tags 
                result["tokens"] = tokens 
                # print(result)
                if index % 7 == 0 or index % 8 ==0:
                    json.dump(result,valid)
                    valid.write("\n")
                elif index % 9 == 0:
                    json.dump(result,test)
                    test.write("\n")
                else:
                    json.dump(result,train)
                    train.write("\n")
                # chunk_tags
                # row["chunk_tags"].split()
                # print(index)

        # print(f"chunk_mapping:{chunk_mapping}") 
        # print(f"chunk_count:{chunk_count}") 
        # print(f"pos_mapping:{pos_mapping}") 
        # print(f"pos_count:{pos_count}") 
            

/homeL/1sliu/anaconda3/envs/jarvix_qa/lib/python3.8/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'zh_core_web_lg' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.3.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
